<a href="https://colab.research.google.com/github/dal7collab/google_collab_py/blob/main/nb_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
train = [('sunny','hot','high','FALSE','N'),
        ('sunny','hot','high','TRUE','N'),
        ('overcast','hot','high','FALSE','P'),
        ('rain','mild','high','FALSE','P'),
        ('rain','cool','normal','FALSE','P'),
        ('rain','cool','normal','TRUE','N'),
        ('overcast','cool','normal','TRUE','P'),
        ('sunny','mild','high','FALSE','N'),
        ('sunny','cool','normal','FALSE','P'),
        ('rain','mild','normal','FALSE','P'),
        ('sunny','mild','normal','TRUE','P'),
        ('overcast','mild','high','TRUE','P'),
        ('overcast','hot','normal','FALSE','P'),
        ('rain','mild','high','TRUE','N')]
        
train = pd.DataFrame(train, columns = ['Outlook','Temperature','Humidity','Windy','Class']) 

In [ ]:
target = 'Class'
train_x = train.drop(columns = [target])
train_y = train[target]

In [ ]:
test = [('rain', 'hot', 'high', 'FALSE')]
test = pd.DataFrame(test, columns = ['Outlook','Temperature','Humidity','Windy']) 

In [ ]:
outlook = train.groupby(['Outlook', 'Class']).size()
outlook

Outlook   Class
overcast  P        4
rain      N        2
          P        3
sunny     N        3
          P        2
dtype: int64

In [ ]:
outlook = pd.crosstab(train['Outlook'], train['Class'], margins = True)
c = outlook.loc['sunny', 'N']
'--------------------------------------------------------------------'
x = train[train['Class'] == 'N']
y = x[x['Outlook'] == 'sunny']
print('P(sunny|N) = ' + str(y.shape[0]) + "/" + str(x.shape[0]))

P(sunny|N) = 3/5


In [ ]:
def P1(target, target_value, train = train):
  target_rows = train[train[target] == target_value]

  n           = target_rows.shape[0]
  d           = train.shape[0]

  p_val =  round(n / d, 4)
  print('P(' + target_value + ') = ' + str(n) + "/" + str(d) + ' = ' + str(p_val))
  return p_val

def P(column, column_value, target, target_value, train = train, laplace = 0.01):
  target_rows = train[train[target] == target_value]
  column_rows = target_rows[target_rows[column] == column_value]

  n           = column_rows.shape[0]
  d           = target_rows.shape[0]

  'Laplace correction'
  if d == 0 :
     n = n + laplace
     d = laplace

  p_val =  round(n / d, 4)
  print('P(' + column_value + '|' + target_value + ') = ' + str(n) + "/" + str(d) + ' = ' + str(p_val))

  return p_val

In [ ]:
p = P('Outlook', 'sunny', 'Class', 'N')

P(sunny|N) = 3/5 = 0.6


In [ ]:
p1 = P('Outlook',     'rain',  'Class', 'P') * \
     P('Temperature', 'hot',   'Class', 'P') * \
     P('Humidity',    'high',  'Class', 'P') * \
     P('Windy',       'FALSE', 'Class', 'P') * \
     P1('Class', 'P')
print(p1)
print('-'*50)
p1 =      P('Outlook',     'rain',  'Class', 'N') 
p1 = p1 * P('Temperature', 'hot',   'Class', 'N') 
p1 = p1 * P('Humidity',    'high',  'Class', 'N') 
p1 = p1 * P('Windy',       'FALSE', 'Class', 'N') 
p1 = p1 * P1('Class', 'N')
print(p1)

P(rain|P) = 3/9 = 0.3333
P(hot|P) = 2/9 = 0.2222
P(high|P) = 3/9 = 0.3333
P(FALSE|P) = 6/9 = 0.6667
P(P) = 9/14 = 0.6429
0.0105800705291164
--------------------------------------------------
P(rain|N) = 2/5 = 0.4
P(hot|N) = 2/5 = 0.4
P(high|N) = 4/5 = 0.8
P(FALSE|N) = 2/5 = 0.4
P(N) = 5/14 = 0.3571
0.018283520000000004


In [ ]:
from sklearn.preprocessing import LabelEncoder

def validate(data, column):
  res = False
  if (data[column].dtype == 'object')  or \
     (data[column].dtype == 'category'):
     res = True
  return res

def fit_label_encoder(data):
  le = {col: LabelEncoder() for col in data.columns}
  ld = {}
  for col in data.columns:
    if validate(data, col):
      le[col].fit(data[col])
      ld[col] = dict(zip(le[col].classes_, le[col].transform(le[col].classes_)))
  return le, ld

def transform_label_encoder(data, label_encoder):
  data_e = data.copy(deep = True)
  for col in data.columns:
    if validate(data, col):
      data_e[col] = label_encoder[col].transform(data[col])  
  return data_e 

def fit_transform_label_encoder(data):
  le, ld = fit_label_encoder(data)
  data_e = transform_label_encoder(data, le)
  return le, ld, data_e

def info_label_encoder(data, label_decoder):
  for col in data.columns:
    if validate(data, col):
      print(col + ':' + str(label_decoder[col]))

In [ ]:
le, ld, train_e = fit_transform_label_encoder(train)
info_label_encoder(train, ld)

test_e = transform_label_encoder(test, le)

print('-' * 50) 
print(train_e)
print('-' * 50)
print(test)
print('-' * 50)
print(test_e)

Outlook:{'overcast': 0, 'rain': 1, 'sunny': 2}
Temperature:{'cool': 0, 'hot': 1, 'mild': 2}
Humidity:{'high': 0, 'normal': 1}
Windy:{'FALSE': 0, 'TRUE': 1}
Class:{'N': 0, 'P': 1}
--------------------------------------------------
    Outlook  Temperature  Humidity  Windy  Class
0         2            1         0      0      0
1         2            1         0      1      0
2         0            1         0      0      1
3         1            2         0      0      1
4         1            0         1      0      1
5         1            0         1      1      0
6         0            0         1      1      1
7         2            2         0      0      0
8         2            0         1      0      1
9         1            2         1      0      1
10        2            2         1      1      1
11        0            2         0      1      1
12        0            1         1      0      1
13        1            2         0      1      0
-----------------------------------

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()
target = 'Class'
train_e_x = train_e.drop(columns = [target])
train_e_y = train_e[target]
clf.fit(train_e_x, train_e_y)
clf

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
print(clf.predict(test_e))
print(clf.predict_proba(test_e))

[0]
[[0.55361246 0.44638754]]


In [ ]:
def one_hot_encoder(train, test = None, target = None, drop = True):
  hc     = '_Train_'
  data_e = train.copy(deep = True)
  data_e[hc] = True

  if test is not None:
    test_e = test.copy(deep = True)
    test_e[target] = None
    test_e[hc]     = False
    data_e = data_e.merge(test_e, how = 'outer')

  columns = []
  for col in train.columns:
    if (col != target) and (col != hc):
      if (train[col].dtype == 'object')  or \
         (train[col].dtype == 'category'):
        columns.append(col)
        temp_e = pd.get_dummies(data_e[col], prefix = col)
        data_e = pd.concat([data_e, temp_e], axis = 1)

  if drop == True:
    data_e = data_e.drop(columns, axis = 1)
    
  if test is not  None:
    train_e = data_e[data_e[hc] == True]
    test_e  = data_e[data_e[hc] == False]
    test_e  = test_e.drop([hc, target], axis = 1)
  else:
    train_e = data_e
    test_e  = None

  train_e  = train_e.drop(hc, axis = 1)
  return train_e, test_e 

In [ ]:
train_e, test_e = one_hot_encoder(train, test, target = 'Class', drop = True)

print(train_e.info())
print(test_e.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Class             14 non-null     object
 1   Outlook_overcast  14 non-null     uint8 
 2   Outlook_rain      14 non-null     uint8 
 3   Outlook_sunny     14 non-null     uint8 
 4   Temperature_cool  14 non-null     uint8 
 5   Temperature_hot   14 non-null     uint8 
 6   Temperature_mild  14 non-null     uint8 
 7   Humidity_high     14 non-null     uint8 
 8   Humidity_normal   14 non-null     uint8 
 9   Windy_FALSE       14 non-null     uint8 
 10  Windy_TRUE        14 non-null     uint8 
dtypes: object(1), uint8(10)
memory usage: 364.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 14 to 14
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Outlook_overcast  1 non-null      uint8
